# Úkol: Zpracování Zoom reportů
Vytvořte Python skript, který zpracuje reporty o připojení účastníků on-line kurzu [Praha Coding](https://www.prahacoding.cz) a vygeneruje individuální report /`csv`/ pro každého z účastníků.   
Vstupní data ze tří různých kurzů jsou v samostatných složkách /označených číslem kurzu/ v souboru `data/source/reporty`.   
Výstupní soubory budou ve složce `data/output/reporty/<id-kurzu>` se jménem účastníka /bez diakritiky a mezer/

## Struktura vstupních dat

Jednotlivé lekce jsou v samostatných souborech s údaji o době připojení každého z účastníků:

| Meeting ID,Topic | Start Time | End Time | User Email | Duration (Minutes) | Participants |
| -------------- | ------------------ | ------- | ---------- | -------- | ------- | 
| 95645604158  |  Datova Analytika  |  2024-03-21 18:18:42  |  2024-03-21 21:45:54  |  info@prahacoding.cz  |  207  |  15  |  

| Name (Original Name)  |  User Email  |  Join Time  |  Leave Time  |  Duration (Minutes)  |  Guest  |  Recording Consent  |  In Waiting Room |
| --------------|------------------|-------|----------|--------|-------|-------|-----|
| Zdeňka Heczková  |    |  2024-03-21 18:18:42  |  2024-03-21 21:45:21  |  207  |  Yes  |  Yes  |  No |
| František Matějíček  |    |  2024-03-21 18:18:54  |  2024-03-21 21:41:39  |  203  |  Yes  |  Yes  |  No |
| Ján Rožďalovský  |    |  2024-03-21 18:19:07  |  2024-03-21 21:45:53  |  207  |  Yes  |  Yes  |  No |
| Ludmila Kozáková  |    |  2024-03-21 18:21:39  |  2024-03-21 21:12:26  |  171  |  Yes  |  Yes  |  No |
| Iveta Poloková  |    |  2024-03-21 18:22:03  |  2024-03-21 21:12:22  |  171  |  Yes  |  Yes  |  No |
| Petra Strnadová  |    |  2024-03-21 18:27:10  |  2024-03-21 21:12:23  |  166  |  Yes  |  Yes  |  No |
| Karel Pavlacký  |    |  2024-03-21 18:29:27  |  2024-03-21 21:45:54  |  197  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:29:32  |  2024-03-21 18:31:16  |  2  |  Yes  |  Yes  |  No |
| Ondřej Koutný  |    |  2024-03-21 18:29:37  |  2024-03-21 21:12:21  |  163  |  Yes  |  Yes  |  No |
| Oskar Koriťák  |    |  2024-03-21 18:29:52  |  2024-03-21 21:12:23  |  163  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:30:22  |  2024-03-21 18:31:10  |  1  |  Yes  |    |  No |
| Hana Füllová  |    |  2024-03-21 18:31:27  |  2024-03-21 19:02:13  |  31  |  Yes  |    |  No |
| Jiří Boháč  |    |  2024-03-21 18:31:50  |  2024-03-21 21:23:23  |  172  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 18:32:08  |  2024-03-21 18:59:35  |  28  |  Yes  |  Yes  |  No |
| Hana Füllová  |    |  2024-03-21 19:02:14  |  2024-03-21 21:12:24  |  131  |  Yes  |    |  No |


## 0. Počáteční Python nezbytnosti

## Importy

In [ ]:
import pandas
import glob
import os


## Konstanty a funkce

In [ ]:
__file__ = __vsc_ipynb_file__
SCRIPT_FOLDER = os.path.dirname(__file__)
PROJECT_FOLDER = os.path.dirname(SCRIPT_FOLDER)
SOURCE_FOLDER = os.path.join(PROJECT_FOLDER,'data','source','reporty','PCS001')
''' slozka se zdrojovymi Zoom reporty '''
DESTINATION_FOLDER = os.path.join(PROJECT_FOLDER,'data','output','reporty','PCS001')
''' slozka s vyslednymi individualnimi reporty '''
ZOOM_REPORTS = "*.csv"
''' maska nazvu zdrojovych souboru '''

def to_ascii(text):
    ''' replace diacritics in text with ASCII
    '''
    chars_from = 'ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝßàáâãäåçèéêëìíîïñòóôõöùúûüýÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĬĭĮįİıĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŉŊŋŌōŎŏŐőŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſ'
    chars_to  =  'AAAAAACEEEEIIIINOOOOOUUUUYsaaaaaaceeeeiiiinooooouuuuyyAaAaAaCcCcCcCcDdDdEeEeEeEeEeGgGgGgGgHhHhIiIiIiIiIiKkkLlLlLlLlLlNnNnNnNnNOoOoOoRrRrRrSsSsSsSsTtTtTtUuUuUuUuUuUuWwYyYZzZzZzs'
    output = ""
    for chr in text:
        pos = chars_from.find(chr)
        if pos < 0:
            output += chr
        else:
            output += chars_to[pos]
    return output


### Sloupce, které nás zajímají

In [ ]:
# z originálu
ZOOM_NAME = 'Name (Original Name)'
ZOOM_JOIN = 'Join Time'
ZOOM_LEAVE = 'Leave Time'

# do výsledného reportu
REPORT_NAME = "Student"
REPORT_JOIN = "Čas připojení"
REPORT_LEAVE = "Čas odpojení"
REPORT_DURATION = "Doba on-line"

# pomocné sloupce
COL_LESSON = 'Lekce'
COL_DURATION = 'Duration'


## 1. Přečti zdrojové soubory a spoj vše do jednoho `DataFrame`

## 2. Krok za krokem vytvoř individuální reporty

### a. přidej sloupec se dnem konání lekce

### b. seskup údaje podle studentů a spoj vícenásobná připojení

### c. spočítej dobu `on-line` v minutách

### d. vytvoř cílovou složku

### e. vytvoř individální reporty